<a href="https://colab.research.google.com/github/cjfergie/Reinforcement-Learning/blob/main/Reinforcement_Learning_Trend_Following.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -nc https://lazyprogrammer.me/course_files/SPY.csv

--2024-03-31 01:12:52--  https://lazyprogrammer.me/course_files/SPY.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 251935 (246K) [text/csv]
Saving to: ‘SPY.csv’

SPY.csv             100%[===================>] 246.03K  --.-KB/s    in 0.06s   

2024-03-31 01:12:53 (3.99 MB/s) - ‘SPY.csv’ saved [251935/251935]



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv('SPY.csv', index_col='Date', parse_dates=True)

In [4]:
#make features
df['FastSMA'] = df['Close'].rolling(16).mean()
df['SlowSMA'] = df['Close'].rolling(33).mean()
feats = ['FastSMA', 'SlowSMA']

In [5]:
df['LogReturn'] = np.log(df['Close']).diff()

In [58]:
#split into train and test
Ntest = 1000
train_data = df.iloc[:-Ntest].copy()
test_data = df.iloc[-Ntest:].copy()

In [59]:
class Env:
  def __init__(self, df):
    self.df = df
    self.n = len(df)
    self.current_idx = 0
    self.action_space = [0, 1, 2] # BUY, SELL, HOLD
    self.invested = 0

    self.states = self.df[feats].to_numpy()
    self.rewards = self.df['LogReturn'].to_numpy()
    self.total_buy_and_hold = 0

  def reset(self):
    self.current_idx = 0
    self.total_buy_and_hold = 0
    self.invested = 0
    return self.states[self.current_idx]

  def step(self, action):
    # need to return (next_state, reward, done)
    self.current_idx += 1
    if self.current_idx >= self.n:
      raise Exception("Episode already done")

    if action == 0: # BUY
      self.invested = 1
    elif action == 1: # SELL
      self.invested = 0

    # compute reward
    if self.invested:
      reward = self.rewards[self.current_idx]
    else:
      reward = 0

    # state transition
    next_state = self.states[self.current_idx]

    # baseline
    self.total_buy_and_hold += self.rewards[self.current_idx]

    # done flag
    done = (self.current_idx == self.n - 1)
    return next_state, reward, done

In [60]:
class Agent:
  def _init_(self):
    self.is_invested = False

  def act(self, state):
    assert(len(state) == 2)
    #(fast, slow)

    if state[0] > state[1] and not self.is_invested:
      self.is_invested = True
      return 0 # buy

    if state[0] < state[1] and not self.is_invested:
      self.is_invested = False
      return 1 # sell

    return 2 # do nothing

In [61]:
def play_one_episode(agent, env):
  state = env.reset()
  done = False
  total_reward = 0
  agent.is_invested = False

  while not done:
    action = agent.act(state)
    next_state, reward, done = env.step(action)
    total_reward += reward

  return total_reward


In [62]:
train_env = Env(train_data)
test_env = Env(test_data)

In [63]:
agent = Agent()

In [64]:
train_reward = play_one_episode(agent, train_env)

In [65]:
test_reward = play_one_episode(agent, test_env)

In [66]:
train_reward, train_env.total_buy_and_hold

(0, 0.5970866514889401)

In [67]:
test_reward, test_env.total_buy_and_hold

(0.19307543946998518, 0.19307543946998518)